In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import re


In [3]:
# Tính toán Jaccard similarity
def jaccard_similarity(query, document):
    query_words = set(re.split(r'\s+', query.lower().strip()))
    doc_words = set(re.split(r'\s+', document.lower().strip()))
    intersection = query_words.intersection(doc_words)
    union = query_words.union(doc_words)
    return len(intersection) / len(union)

In [4]:
df = pd.read_csv('questions_answers.csv')

# Tách câu hỏi và câu trả lời
questions = df['question']
answers = df['answer']
print(questions)

0      Nguyên tắc xét tuyển của học viện năm nay ntn?...
1                       Câu hỏi thường gặp về tuyển sinh
2      Chương trình đào tạo tại miền Nam có khác với ...
3      Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...
4      Em đăng ký NV1 là ngành An toàn thông tin, NV2...
                             ...                        
866    Điểm chuẩn ngành công nghệ internet vạn vật nă...
867    Điểm chuẩn ngành công nghệ internet vạn vật nă...
868    Điểm chuẩn ngành công nghệ internet vạn vật nă...
869    Điểm chuẩn ngành công nghệ internet vạn vật nă...
870    Chỉ tiêu ngành công nghệ internet vạn vật năm ...
Name: question, Length: 871, dtype: object


In [5]:
# Tạo bộ từ điển các từ viết tắt
abbreviation_dict = {
    "cntt": "công nghệ thông tin",
    "attt": "an toàn thông tin",
    "iot": "công nghệ internet vạn vật",
    "fintech": "công nghệ tài chính",
    "cndpt": "công nghệ đa phương tiện",
    "ttdpt": "truyền thông đa phương tiện",
    "qtkd": "quản trị kinh doanh",
    "tmdt": "thương mại điện tử",
    "khmt": "khoa học máy tính",
    "clc": "chất lượng cao",
}

# Hàm để thay thế các từ viết tắt trong câu bằng từ đầy đủ
def replace_abbreviations(text, abbreviation_dict):
    words = text.split()
    new_words = []
    for word in words:
        if word in abbreviation_dict:
            new_words.append(abbreviation_dict[word])
        else:
            new_words.append(word)
    return ' '.join(new_words)

# Ví dụ sử dụng
user_input = "Tôi đang học cntt và muốn tìm hiểu về ngành clc"
processed_input = replace_abbreviations(user_input, abbreviation_dict)

print("Trước khi xử lý:", user_input)
print("Sau khi xử lý:", processed_input)


Trước khi xử lý: Tôi đang học cntt và muốn tìm hiểu về ngành clc
Sau khi xử lý: Tôi đang học công nghệ thông tin và muốn tìm hiểu về ngành chất lượng cao


In [6]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

remove_punctuation("Chi tiết về     Xét tuyển tài năng?@")

'Chi tiết về     Xét tuyển tài năng'

In [7]:
def to_lowercase(text):
    return text.lower()
to_lowercase('Chi tiết về     Xét tuyển tài năng')


'chi tiết về     xét tuyển tài năng'

In [8]:
def replace_comma(text):
    return " ".join(text.replace(',', ' ').split())
replace_comma('chi tiết về     xét tuyển tài năng năm 2021, 2022,2023')


'chi tiết về xét tuyển tài năng năm 2021 2022 2023'

In [9]:
from underthesea import word_tokenize
def tokenizerText(text):
    return word_tokenize(text, format="text")
tokenizerText('chi tiết về     xét tuyển tài năng năm 2021  2022 2023')

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'chi_tiết về xét_tuyển tài_năng năm 2021 2022 2023'

In [10]:
text = "hàng kém chất lượng,chăn đắp lên dính lông lá khắp người. thất vọng"
tokenizerText(text)

'hàng kém chất_lượng , chăn đắp lên dính lông lá khắp người . thất_vọng'

In [11]:
from underthesea import text_normalize
text_normalize("Ðảm baỏ chất lựơng phòng thí nghịêm hoá học")

'Đảm bảo chất lượng phòng thí nghiệm hóa học'

In [12]:
# Khởi tạo tập hợp để lưu trữ các từ dừng
stopwords = set()

# Đọc từng hàng trong file stopwords.txt
with open('stopwords.txt', 'r', encoding='utf-8') as fp:
    for line in fp:
        word = line.strip() # Loại bỏ khoảng trắng đầu và cuối dòng
        if word:  # Kiểm tra xem dòng không rỗng
            stopwords.add(word)

# In danh sách các từ dừng
print(stopwords)
print(len(stopwords))


{'dù_gì', 'biết_mình', 'biết_thế', 'cho_về', 'biết', 'là_phải', 'là_thế_nào', 'bao_nhiêu', 'là_là', 'cho_rằng', 'biết_đâu_chừng', 'bởi', 'dù_rằng', 'cho_tin', 'bao_lâu', 'bởi_vậy', 'biết_đâu_đấy', 'cho_rồi', 'bao_giờ', 'biết_chừng_nào', 'biết_trước', 'bị', 'bởi_chưng', 'cho_nên', 'ngay_thật', 'mấy', 'ngay_lúc_này', 'biết_chắc', 'bỗng_nhưng', 'hỏi', 'bởi_vì', 'bỗng_nhiên', 'làm_cho', 'là_cùng', 'bỗng_chốc', 'bỗng', 'bỗng_không', 'cho_tới_khi', 'bởi_thế_cho_nên', 'cho_nhau', 'ngay_tức_khắc', 'bấy_nhiêu', 'cho_chắc', 'biết_bao_nhiêu', 'tao', 'tôi', 'cho_được', 'bây_giờ', 'bởi_tại', 'biết_mấy', 'bỗng_đâu', 'cho_ăn', 'ngay_tức_thì', 'bây_nhiêu', 'cho_đến_khi', 'biết_đâu', 'bởi_ai', 'cho_đang', 'là_ít', 'về', 'cho_đến_nỗi', 'bởi_sao', 'ngay_cả', 'bởi_nhưng', 'bấy_lâu', 'là_nhiều', 'làm_bằng', 'dù_dì', 'cho', 'ngay_khi_đến', 'biết_được', 'bởi_đâu', 'bỗng_thấy', 'làm', 'ngay_khi', 'ngay_lúc', 'cho_đến', 'dù_cho', 'bấy_lâu_nay', 'biết_bao', 'dù_sao', 'bởi_thế', 'bỗng_dưng', 'bấy_giờ', 'là_vì', 

In [13]:
def remove_stopwords(line):
    words = []
    for word in line.split():
        if word not in stopwords:
            words.append(word)
    return ' '.join(words)
text = 'Điểm chuẩn ngành cntt theo phương thức thi thpt 2023 là bao nhiêu?'
print(tokenizerText(text))
print(remove_stopwords(tokenizerText(text)))

Điểm chuẩn ngành cntt theo phương_thức thi thpt 2023 là bao_nhiêu ?
Điểm chuẩn ngành cntt theo phương_thức thi thpt 2023 ?


In [14]:
def processing_text_for_db(text):
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_comma(text)
    text = tokenizerText(text)
    text = remove_stopwords(text)
    return text

In [17]:
def processing_text_for_db_rerank(text):
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_comma(text)
    return text
print(processing_text_for_db_rerank("Điểm chuẩn ngành Công Nghệ    thông tin năm 2024?"))
print(processing_text_for_db("Điểm chuẩn ngành Công Nghệ    thông tin năm 2024?"))

điểm chuẩn ngành công nghệ thông tin năm 2024
điểm chuẩn ngành công_nghệ_thông_tin năm 2024


In [53]:
def processing_text_for_query(text):
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_comma(text)
    text = replace_abbreviations(text, abbreviation_dict)
    text = tokenizerText(text)
    text = remove_stopwords(text)
    return text

Demo preprocessing question

In [54]:
q = "Điểm chuẩn ngành cntt clc năm 2023 theo phương thức thi thpt tại cơ sở miền bắc   là bao   nhiêu ?!$"
q1 = "mã ngành của ngành qtkd là bao nhiêu."
print(processing_text_for_query(q))
print(processing_text_for_query(q1))



điểm chuẩn ngành công_nghệ_thông_tin chất_lượng cao năm 2023 theo phương_thức thi thpt tại cơ_sở miền bắc
mã ngành của ngành quản_trị kinh_doanh


In [55]:
processed_questions = []
for question in questions:  # Sử dụng tqdm để theo dõi tiến trình nếu cần
    # Áp dụng các hàm tiền xử lý
    processed_text = processing_text_for_db(question)
   
    # Lưu kết quả vào list processed_questions
    processed_questions.append(processed_text)

# Thêm cột mới vào DataFrame hoặc ghi đè lên cột 'question' hiện tại
df['processed_question'] = processed_questions

print(len(df))
# In ra vài dòng đầu của DataFrame sau khi đã tiền xử lý
print(df.head())

858
                                            question  \
0  Nguyên tắc xét tuyển của học viện năm nay ntn?...   
1                   Câu hỏi thường gặp về tuyển sinh   
2  Chương trình đào tạo tại miền Nam có khác với ...   
3  Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...   
4  Em đăng ký NV1 là ngành An toàn thông tin, NV2...   

                                              answer  \
0  I. Xét tuyển dựa vào kết quả thi THPT 2024:\n-...   
1                            Bạn đang gặp vấn đề gì?   
2  Chương trình đào tạo của Học viện được áp dụng...   
3  Chỉ tiêu đào tạo của từng cơ sở đã được Học vi...   
4  Không được em nhé! Theo nguyên tắc xét tuyển t...   

                                  processed_question  
0  nguyên_tắc xét_tuyển của học_viện năm nay ntn ...  
1                          câu thường gặp tuyển_sinh  
2  chương_trình đào_tạo tại miền nam có khác với ...  
3  trường mình có 2 cơ_sở vậy sẽ phân_bố học_sinh...  
4  em đăng_ký nv1 ngành an_toàn thông_tin nv2 n

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Function to encode each question into vector representation
def encode_question(question):
   # Tokenize và chuyển đổi câu văn bản thành tensor
    tokens = tokenizer(question, return_tensors='pt', padding=True, truncation=True)

    # Sử dụng model để mã hóa câu văn bản thành vector
    with torch.no_grad():
        output = phobert(**tokens)

    # Lấy vector biểu diễn từ outputs của model
    return output.last_hidden_state.mean(dim=1).numpy()  # Trung bình các vector token




Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [57]:
df['question_vector'] = df['processed_question'].apply(encode_question)
print(df.head())


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                            question  \
0  Nguyên tắc xét tuyển của học viện năm nay ntn?...   
1                   Câu hỏi thường gặp về tuyển sinh   
2  Chương trình đào tạo tại miền Nam có khác với ...   
3  Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...   
4  Em đăng ký NV1 là ngành An toàn thông tin, NV2...   

                                              answer  \
0  I. Xét tuyển dựa vào kết quả thi THPT 2024:\n-...   
1                            Bạn đang gặp vấn đề gì?   
2  Chương trình đào tạo của Học viện được áp dụng...   
3  Chỉ tiêu đào tạo của từng cơ sở đã được Học vi...   
4  Không được em nhé! Theo nguyên tắc xét tuyển t...   

                                  processed_question  \
0  nguyên_tắc xét_tuyển của học_viện năm nay ntn ...   
1                          câu thường gặp tuyển_sinh   
2  chương_trình đào_tạo tại miền nam có khác với ...   
3  trường mình có 2 cơ_sở vậy sẽ phân_bố học_sinh...   
4  em đăng_ký nv1 ngành an_toàn thông_tin nv2 

In [58]:
query = "Điểm chuẩn ngành an toàn thông tin theo phương thức thi thpt 2023 miền bắc?"
processed_query = processing_text_for_query(query)
print(processed_query)
query_vector = encode_question(processed_query)
print(query_vector.shape)
print(query_vector)


điểm chuẩn ngành an_toàn thông_tin theo phương_thức thi thpt 2023 miền bắc
(1, 768)
[[ 2.33505383e-01  9.27157551e-02 -1.87626928e-01 -1.44157320e-01
  -1.40480623e-01  1.02259435e-01  5.98974600e-02  1.39165312e-01
  -1.60667598e-01  8.39552060e-02 -1.43745929e-01  1.78501070e-01
   1.47883054e-02 -1.41717300e-01 -5.08329198e-02  3.44799161e-02
  -7.32005313e-02  2.64150113e-01  2.00316235e-01 -1.04688317e-01
  -1.74132988e-01  1.70892015e-01  5.68007939e-02  3.94880831e-01
  -1.37368724e-01 -1.05723284e-01  5.83432661e-03  1.82094187e-01
  -1.40878633e-01 -1.37640446e-01 -1.15896396e-01 -2.41887599e-01
  -1.12938456e-01  2.01233402e-01  3.76158327e-01  4.25440669e-01
  -6.14149757e-02 -4.31861244e-02  1.69253424e-01  3.26822139e-02
  -9.64203756e-03  2.82292366e-01  1.75632238e-02 -9.13411677e-02
   2.41848126e-01  1.47984669e-01  3.81541848e-02  1.04409389e-01
   2.32662205e-02  2.03086957e-01  2.37838641e-01  2.93926030e-01
   5.60786799e-02  3.86773139e-01  8.71510431e-02  1.71963

In [59]:
query = "Điểm chuẩn ngành cntt năm 2023 cntt năm 2023 cntt năm 2023 cntt năm 2023 miền nam, bắc, trung theo thpt tại miền bắc "
processed_query = processing_text_for_query(query)
print(processed_query)
query_vector = encode_question(processed_query)

cosine_similarities = [cosine_similarity(
        query_vector, qv).flatten() for qv in df['question_vector']]

best_match_index = np.argmax(cosine_similarities)
print(np.max(cosine_similarities))



điểm chuẩn ngành công_nghệ_thông_tin năm 2023 công_nghệ_thông_tin năm 2023 công_nghệ_thông_tin năm 2023 công_nghệ_thông_tin năm 2023 miền nam bắc trung theo thpt tại miền bắc
0.87668


In [60]:
from transformers import pipeline

corrector = pipeline("text2text-generation", model="bmd1905/vietnamese-correction")


In [61]:
from model import correction_model
# Example
MAX_LENGTH = 512

# Define the text samples
texts = [
    "điểm chuẩn ngành coongg nghệ thông tin namw 2024 2025 ",
    

]

# Batch prediction
predictions = correction_model(texts, MAX_LENGTH)

# Print predictions
for text, pred in zip(texts, predictions):
    print("- " + pred['generated_text'])


- Điểm chuẩn ngành công nghệ thông tin năm 2024 2025 .


In [62]:
# import pandas as pd
# import numpy as np
# df = pd.read_csv('embeddings.csv', converters={
#     'vector_embeddings': lambda x: np.fromstring(x.strip('[]'), sep=' ')
# })
# embeddings = np.vstack(df['vector_embeddings'])

In [63]:
# embeddings.shape


In [64]:
# from qdrant_client import QdrantClient
# from qdrant_client.models import PointStruct,VectorParams

# # Kết nối tới Qdrant
# client = QdrantClient(":memory:")
# # Cấu hình vector
# vectors_config = VectorParams(size=embeddings.shape[1], distance="Cosine")
# # Tạo một collection trong Qdrant
# collection_name = "embeddings_qa"
# client.recreate_collection(collection_name=collection_name, vectors_config=vectors_config)

# # Kiểm tra xem collection có tồn tại hay không
# collections = client.get_collections().collections
# collection_names = [col.name for col in collections]
# print("Available collections:", collection_names)

# if collection_name not in collection_names:
#     raise ValueError(f"Collection {collection_name} not found after creation.")
# # Chuẩn bị các điểm (points) để thêm vào Qdrant
# points = [
#     PointStruct(id=i, vector=embedding.tolist(), payload={"text": df['question'][i]})
#     for i, embedding in enumerate(embeddings)
# ]

# # Thêm các điểm vào Qdrant
# client.upsert(collection_name=collection_name, points=points)


In [65]:
# from model import embeddings
# from preprocess import processing_text_for_query

# # Kết nối tới Qdrant
# # client = QdrantClient(":memory:")
# # Tạo embedding cho câu truy vấn
# query_sentence = "học phí ngành cntt"
# query_sentence = processing_text_for_query(query_sentence)
# query_embedding=embeddings(query_sentence)

# # Truy vấn Qdrant
# search_result = client.search(
#     collection_name='embeddings_qa',
#     query_vector=query_embedding.tolist()[0],  # Chuyển đổi thành danh sách
#     limit=10  # Số lượng kết quả trả về
# )

# # In ra các câu gốc tương ứng với các kết quả tìm được
# for result in search_result:
#     print(f"ID: {result.id}")
#     print(f"Score: {result.score}")
#     print(f"Payload: {result.payload}")
#     print("---")


In [20]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
        'jinaai/jina-reranker-v2-base-multilingual',
        torch_dtype="auto",
        trust_remote_code=True,
)


In [25]:
    # Example query and documents
query =  "điểm chuẩn ngành marketing năm 2024 theo phương thức thi đánh giá năng lực"

documents = [
    "điểm chuẩn ngành marketing theo phương thức thi đánh giá năng lực năm 2024 tại cơ sở miền nam là bao nhiêu",
    "điểm chuẩn ngành marketing theo phương thức thi đánh giá năng lực năm 2024 tại cơ sở miền bắc là bao nhiêu",
    "điểm chuẩn ngành marketing chất lượng cao  theo phương thức thi đánh giá năng lực năm 2024 tại cơ sở miền nam là bao nhiêu",
    "Thời tiết hôm nay rất đẹp",
    "Trời mưa to quá",
    "Hôm nay trời đẹp quá",
    ]
    # construct sentence pairs
sentence_pairs = [[query, doc] for doc in documents]

scores = model.compute_score(sentence_pairs, max_length=1024)
print("Scores:\n")
print(scores)
result = model.rerank(
              query,
              documents,
              max_query_length=512,
              max_length=1024,
            top_n=3
)
print(result)

print(result[0]['document'])
    


Scores:

[0.6976089477539062, 0.700894832611084, 0.7233095169067383, 0.037892427295446396, 0.05749328061938286, 0.08756383508443832]
[{'document': 'điểm chuẩn ngành marketing chất lượng cao  theo phương thức thi đánh giá năng lực năm 2024 tại cơ sở miền nam là bao nhiêu', 'relevance_score': 0.7233095169067383, 'index': 2}, {'document': 'điểm chuẩn ngành marketing theo phương thức thi đánh giá năng lực năm 2024 tại cơ sở miền bắc là bao nhiêu', 'relevance_score': 0.700894832611084, 'index': 1}, {'document': 'điểm chuẩn ngành marketing theo phương thức thi đánh giá năng lực năm 2024 tại cơ sở miền nam là bao nhiêu', 'relevance_score': 0.6976089477539062, 'index': 0}]
điểm chuẩn ngành marketing chất lượng cao  theo phương thức thi đánh giá năng lực năm 2024 tại cơ sở miền nam là bao nhiêu


In [68]:
from sentence_transformers import CrossEncoder

questions_vector = df['question_vector']
processed_questions = df['processed_question']  
questions = df['question']
def get_response(user_query):
    processed_query = processing_text_for_query(user_query)
    query_vector = encode_question(processed_query)
    # Tính toán độ tương đồng cosine giữa câu truy vấn của người dùng và các câu hỏi trong database
    cosine_similarities = [cosine_similarity(
        query_vector, qv).flatten() for qv in questions_vector]
    jaccard_similarities = [jaccard_similarity(
        processed_query, q) for q in  df['processed_question']]

    # Kết hợp kết quả từ hai độ tương đồng
    alpha = 0.6
    beta = 0.4
    # Chuyển đổi jaccard_similarities thành mảng một chiều
    jaccard_array = np.array(jaccard_similarities).reshape(len(df['processed_question']),1)

    documents = []
    pos = []
    # Kết hợp điểm số từ hai độ tương đồng
    combined_scores = alpha * np.array(cosine_similarities) + beta * jaccard_array
    sorted_indices = np.argsort(combined_scores,axis=0)
    n = 5
    largest_indices = sorted_indices[::-1][:n]
    for i in largest_indices:
        documents.append(processing_text_for_db_rerank(questions[int(i)]))
        pos.append([int(i)])
        print(int(i))
    query_rerank = processing_text_for_db_rerank(user_query)
    model = CrossEncoder(
        "jinaai/jina-reranker-v2-base-multilingual",
        automodel_args={"torch_dtype": "auto"},
        trust_remote_code=True,
    )
    # construct sentence pairs
    # sentence_pairs = [[query_rerank, doc] for doc in documents]
    # scores = model.predict(sentence_pairs, convert_to_tensor=True).tolist()

    rankings = model.rank(query_rerank, documents, return_documents=True, convert_to_tensor=True)
    print(f"Query: {query_rerank}")
    for ranking in rankings:
        print(f"ID: {ranking['corpus_id']}, Score: {ranking['score']:.4f}, Text: {ranking['text']}")
    print(rankings[0]['corpus_id'])
    print(answers[pos[rankings[0]['corpus_id']]])
    best_match_index = np.argmax(combined_scores)
    if np.max(combined_scores) > 0.6:
        print(np.max(combined_scores))
        print(processed_query)
        print(processed_questions[best_match_index])
        print(questions[best_match_index])
        return answers[best_match_index]
    else:
        print(np.max(combined_scores))
        return "Tôi không hiểu câu hỏi của bạn. Vui lòng đặt câu hỏi đầy đủ hơn."
user_input = "chỉ tiêu ngành công nghệ thông tin năm 2024"
response = get_response(user_input)
print(response)



61
195
151
181
237


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Query: chỉ tiêu ngành công nghệ thông tin năm 2024
ID: 0, Score: 0.8984, Text: chỉ tiêu của ngành công nghệ thông tin năm 2024 là bao nhiêu
ID: 1, Score: 0.8984, Text: chỉ tiêu của ngành công nghệ thông tin năm 2024 là bao nhiêu
ID: 2, Score: 0.8633, Text: chỉ tiêu của ngành công nghệ thông tin chất lượng cao năm 2024 là bao nhiêu
ID: 3, Score: 0.7578, Text: chỉ tiêu của ngành công nghệ thông tin việt nhật năm 2024 là bao nhiêu
ID: 4, Score: 0.3066, Text: chỉ tiêu của ngành kế toán năm 2024 là bao nhiêu
0
61    Bạn muốn hỏi chỉ tiêu dựa trên phương thức xét...
Name: answer, dtype: object
0.8921944697697958
chỉ_tiêu ngành công_nghệ_thông_tin năm 2024
chỉ_tiêu của ngành công_nghệ_thông_tin năm 2024
Chỉ tiêu của Ngành Công nghệ thông tin năm 2024 là bao nhiêu?
Bạn muốn hỏi chỉ tiêu dựa trên phương thức xét tuyển nào, cơ sở miền bắc hay nam? Ví dụ về phương thức xét tuyển như xét tuyển kết hợp, xét tuyển tài năng, thi thpt, đánh giá năng lực, đánh giá tư duy.


In [48]:
from sentence_transformers import CrossEncoder
import pandas as pd
import json
import numpy as np
from preprocess import processing_text_for_query, processing_text_for_db_rerank, processing_text_for_query_rerank
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'jinaai/jina-reranker-v2-base-multilingual',
    torch_dtype="auto",
    trust_remote_code=True,
)

# corrector = pipeline("text2text-generation",
#                          model="bmd1905/vietnamese-correction")
# def correction_model(texts):
#     return corrector(texts, max_length=512)

# Đọc DataFrame từ file CSV
dataFrame = pd.read_csv('embeddings.csv')
# Chuyển đổi chuỗi JSON trở lại danh sách và sau đó thành mảng NumPy
dataFrame['vector_embeddings'] = dataFrame['vector_embeddings'].apply(lambda x: np.array(json.loads(x)))
questions_vector = dataFrame['vector_embeddings']
processed_questions = dataFrame['processed_question']  
questions = dataFrame['question']
answers = dataFrame['answer']

def get_response(user_query):
    processed_query = processing_text_for_query(user_query)
    query_vector = encode_question(processed_query)
    # Tính toán độ tương đồng cosine giữa câu truy vấn của người dùng và các câu hỏi trong database
    cosine_similarities = [cosine_similarity(
        query_vector, qv).flatten() for qv in questions_vector]
    jaccard_similarities = [jaccard_similarity(
        processed_query, q) for q in  dataFrame['processed_question']]

    # Kết hợp kết quả từ hai độ tương đồng
    alpha = 0.6
    beta = 0.4
    # Chuyển đổi jaccard_similarities thành mảng một chiều
    jaccard_array = np.array(jaccard_similarities).reshape(len(dataFrame['processed_question']),1)

    # Kết hợp điểm số từ hai độ tương đồng
    combined_scores = alpha * np.array(cosine_similarities) + beta * jaccard_array
    sorted_indices = np.argsort(combined_scores,axis=0)
    
    documents = []
    pos = []
    n = 10
    largest_indices = sorted_indices[::-1][:n]
    for i in largest_indices:
        documents.append(processing_text_for_db_rerank(questions[int(i)]))
        pos.append([int(i)])
    print(documents)
    query_rerank = processing_text_for_query_rerank(user_query)
    print(query_rerank)
    result = model.rerank(
        query_rerank,
        documents,
        max_query_length=512,
        max_length=1024,
        top_n=5
    )
    print(result)
    if result[0]['relevance_score'] > 0.6:
        print((result[0]['relevance_score']))
        print(pos[result[0]['index']][0])
        return answers[pos[result[0]['index']][0]]
    else:
        return "Tôi không hiểu câu hỏi của bạn. Vui lòng đặt câu hỏi đầy đủ hơn."
    
global previous_questions
previous_questions = []


def handle_user_question(question):
    try:
        response = get_response(question)
        if response == "Tôi không hiểu câu hỏi của bạn. Vui lòng đặt câu hỏi đầy đủ hơn.":
            for i in range(1, min(3, len(previous_questions) + 1)):
                last_question = previous_questions[-i]
                combined_question = last_question + " " + question
                response = get_response(combined_question)
                if response != "Tôi không hiểu câu hỏi của bạn. Vui lòng đặt câu hỏi đầy đủ hơn.":
                    return response

        # Thêm câu hỏi hiện tại vào danh sách câu hỏi trước đó
        previous_questions.append(question)
        return response
    except (Exception):
        return "Tôi không hiểu câu hỏi của bạn. Vui lòng đặt câu hỏi đầy đủ hơn."
user_input = "ngành qtkd có đào tạo ở miền nam không"
response = get_response(user_input)
print(response)



['ngành quản trị kinh doanh được đào tạo ở cơ sở miền nam không', 'ngành quản trị kinh doanh được đào tạo ở cơ sở miền bắc không', 'ngành kế toán được đào tạo ở cơ sở miền nam không', 'ngành marketing được đào tạo ở cơ sở miền nam không', 'ngành công nghệ thông tin được đào tạo ở cơ sở miền nam không', 'ngành thương mại điện tử được đào tạo ở cơ sở miền nam không', 'ngành khoa học máy tính được đào tạo ở cơ sở miền nam không', 'ngành an toàn thông tin được đào tạo ở cơ sở miền nam không', 'ngành kỹ thuật điện tử viễn thông được đào tạo ở cơ sở miền nam không', 'ngành truyền thông đa phương tiện được đào tạo ở cơ sở miền nam không']
ngành quản trị kinh doanh có đào tạo ở miền nam không


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


[{'document': 'ngành quản trị kinh doanh được đào tạo ở cơ sở miền nam không', 'relevance_score': 0.9273632764816284, 'index': 0}, {'document': 'ngành quản trị kinh doanh được đào tạo ở cơ sở miền bắc không', 'relevance_score': 0.7799928784370422, 'index': 1}, {'document': 'ngành marketing được đào tạo ở cơ sở miền nam không', 'relevance_score': 0.6469064354896545, 'index': 3}, {'document': 'ngành thương mại điện tử được đào tạo ở cơ sở miền nam không', 'relevance_score': 0.46587347984313965, 'index': 5}, {'document': 'ngành kế toán được đào tạo ở cơ sở miền nam không', 'relevance_score': 0.38676342368125916, 'index': 2}]
0.9273632764816284
348
Ngành Quản trị kinh doanh được đào tạo ở cơ sở Hà Nội và Tp Hồ Chí Minh


In [44]:
from sentence_transformers import CrossEncoder
import pandas as pd
import json
import numpy as np
from preprocess import processing_text_for_query, processing_text_for_db_rerank, processing_text_for_query_rerank
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("bmd1905/vietnamese-correction")
model = AutoModelForSeq2SeqLM.from_pretrained("bmd1905/vietnamese-correction")




# Đọc DataFrame từ file CSV
dataFrame = pd.read_csv('embeddings.csv')
# Chuyển đổi chuỗi JSON trở lại danh sách và sau đó thành mảng NumPy
dataFrame['vector_embeddings'] = dataFrame['vector_embeddings'].apply(lambda x: np.array(json.loads(x)))
questions_vector = dataFrame['vector_embeddings']
processed_questions = dataFrame['processed_question']  
questions = dataFrame['question']
answers = dataFrame['answer']

def get_response(question):
    user_query_correct = correction_model(question)
    user_query = user_query_correct[0]['generated_text']
    print(user_query_correct[0]['generated_text'])
    processed_query = processing_text_for_query(user_query)
    
    return processed_query

user_input = "chỉ tiêuu ngành cntt namw nay theo phương thức thi đánh giá năng lực"
response = get_response(user_input)
print(response)




Chỉ tiêu ngành CNTT năm nay theo phương thức thi đánh giá năng lực.
chỉ_tiêu ngành công_nghệ_thông_tin năm 2024 theo phương_thức thi đánh_giá năng_lực
